In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 12

In [41]:
Results = pd.read_csv('TestResults.csv', index_col=None).iloc[:,1:]
v = Results['PrimaryDiag'].value_counts() < 4
Results.loc[Results['PrimaryDiag'].isin(v.index[v]), 'PrimaryDiag'] = 'Other'

In [42]:
Results['PrimaryDiag'].value_counts()

G31      147
G30       76
Other     66
I50       64
I25       32
J44       28
M62       21
C34       21
G20       16
I69       13
C18       13
N18       13
J96       12
C25        8
C50        8
C67        5
C90        5
C61        5
E43        4
C56        4
J84        4
I42        4
I67        4
K72        4
Name: PrimaryDiag, dtype: int64

In [80]:
import plotly.express as px


fig = px.bar(Results,x='PrimaryDiag', y=['TotalLos','Pred_Los'],hover_data=['EntitySys'], 
             barmode="group",
             labels=dict(value="Cumulative LOS", PrimaryDiag="Primary Diagnosis", variable="Legend",
                        TotalLos='Actual',Pred_Los="Predicted"),
             title="Comparison of Total Los vs Predicted Los",
             category_orders={"Legend": ["Actual","Predicted"]},
            )
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99))
fig.show()

In [74]:
fig = px.scatter(Results,x='LosRange', y='Pred_Los',hover_data=['EntitySys','PrimaryDiag'])
fig.show()

In [45]:
def show_pred_by_catg(col):
    gdf=Results.groupby([col,'LosRange']).size().reset_index()
    gdf.columns=[col,'LosRange','Patients']
    gdf['MeanLos']=pd.DataFrame(Results.groupby([col,'LosRange'])['TotalLos'].agg(MeanLos='mean').reset_index())['MeanLos']
    gdf['MeanPred']=pd.DataFrame(Results.groupby([col,'LosRange'])['Pred_Los'].agg(MeanPred='mean').reset_index())['MeanPred']
    gdf

    fig = px.scatter(gdf,x='MeanLos', y='MeanPred',hover_data=['Patients',col], size='Patients', color=col,
                    title="Prediction variation by " + col)
    fig.update_layout(
            shapes=[
                dict(
                    type= 'line',
                    yref= 'y', y0=0, y1= np.max(gdf['MeanLos']),
                    xref= 'x', x0=0, x1= np.max(gdf['MeanLos'])
                )
            ])
    fig.show()
show_pred_by_catg('PrimaryDiag')


In [46]:
show_pred_by_catg('Ethinicity')

In [47]:
show_pred_by_catg('MaritalStatus')

In [48]:
show_pred_by_catg('Religion')

In [49]:
show_pred_by_catg('DiagGroup')

In [50]:
show_pred_by_catg('DiagnosisCategory')

In [51]:
Results['AgeInYears']=(Results['AgeAtAdmit'].values/12).astype('int')
show_pred_by_catg('AgeInYears')